In [6]:
import cv2

class VehicleTracker:
    def __init__(self):
        self.background_subtractor = cv2.createBackgroundSubtractorMOG2()
        self.prev_frame = None

    def track_vehicle(self, frame):
        # 배경 제거 수행
        fg_mask = self.background_subtractor.apply(frame)

        # 이전 프레임이 없는 경우 현재 프레임을 이전 프레임으로 설정하고 반환
        if self.prev_frame is None:
            self.prev_frame = frame
            return False

        # 현재 프레임과 이전 프레임의 차이를 계산
        frame_diff = cv2.absdiff(frame, self.prev_frame)
        diff_gray = cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY)
        _, diff_thresh = cv2.threshold(diff_gray, 30, 255, cv2.THRESH_BINARY)

        # 차이가 있는 영역의 개수를 계산
        num_diff_pixels = cv2.countNonZero(diff_thresh)

        # 차이가 있는 영역이 일정 개수 이상인 경우 차량이 움직인 것으로 판단
        if num_diff_pixels > 5000:  # 적절한 임계값을 설정해야 합니다.
            return True
        else:
            return False

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    tracker = VehicleTracker()

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        
        # 차량 추적 및 이동 상태 판단
        is_moving = tracker.track_vehicle(frame)
        driving_status = "driving" if is_moving else "stop"
        
        # 주행 상태를 화면에 표시
        cv2.putText(frame, f"status: {driving_status}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # 결과 출력
        cv2.imshow("Frame", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV"
    main(video_path)


In [9]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image

class Detector:
    def __init__(self, model_path, confidence_threshold=0.5):
        self.model = torch.load(model_path)
        self.model.eval()
        self.confidence_threshold = confidence_threshold
        self.transform = transforms.Compose([
            transforms.Resize((416, 416)),
            transforms.ToTensor(),
        ])

    def detect_objects(self, frame):
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        image_tensor = self.transform(image).unsqueeze(0)
        with torch.no_grad():
            outputs = self.model(image_tensor)
        boxes = []
        for output in outputs:
            for detection in output:
                class_id = int(detection[5])
                confidence = detection[4]
                if confidence > self.confidence_threshold:
                    boxes.append(detection.tolist())
        return boxes

def main(video_path, model_path):
    cap = cv2.VideoCapture(video_path)
    detector = Detector(model_path)

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        
        # 객체 감지 수행
        boxes = detector.detect_objects(frame)

        # 이동 여부 판단
        is_moving = len(boxes) > 0
        driving_status = "주행 중" if is_moving else "정지"
        
        print(f"현재 주행 상태: {driving_status}")
        
        # 결과 출력
        cv2.imshow("Frame", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV"
    model_path = "/home/min/dev_ws/machine_learning/project_3rd/src/model_learning/yolo_detection/all_best.pt"
    main(video_path, model_path)


AttributeError: 'dict' object has no attribute 'eval'